# Pattern matching exercise

## 1. Zaimplementuj algorytmy wyszukiwania wzorców:

---
### a) Naive algorithm

In [1]:
def naive_string_matching(text, pattern):
    txt_len, pat_len = len(text), len(pattern)
    result = []
    for s in range(txt_len - pat_len + 1):
        if pattern == text[s:s+pat_len]:
            result.append(s)
    return result

---
### b) Finite automation algorithm

In [2]:
def transition_table(pattern):
    alphabet = set(pattern)
    ptt_len = len(pattern)
    result = []
    for q in range(ptt_len+1):
        result.append({})
        for l in alphabet:
            k = min(len(pattern), q+1)
            while True:
                if k == 0 or pattern[:k] == (pattern[:q] + l)[-k:]:
                    break
                k -= 1
            result[q][l] = k
    return result            

In [3]:
def fa_string_matching(text, pattern):
    q = 0
    delta = transition_table(pattern)
    txt_len = len(text)
    result = []
    for s in range(txt_len):
        if text[s] in delta[q]:
            q = delta[q][text[s]]
            if q == len(delta) - 1:
                result.append(s+1-q)
        else:
            q = 0
    return result

---
### c) KMP (Knuth–Morris–Pratt) algorithm

In [4]:
def prefix_function(pattern):
    pat_len = len(pattern)
    pi = [0]
    k = 0
    for q in range(1, pat_len):
        while k > 0 and pattern[k] != pattern[q]:
            k = pi[k-1]
        if pattern[k] == pattern[q]:
            k += 1
        pi.append(k)
    return pi

In [5]:
def kmp_string_matching(text, pattern):
    txt_len, pat_len = len(text), len(pattern)
    pi = prefix_function(pattern)
    q = 0
    result = []
    for i in range(txt_len):
        while q > 0 and pattern[q] != text[i]:
            q = pi[q-1]
        if pattern[q] == text[i]:
            q += 1
        if q == pat_len:
            result.append(i - q + 1)
            q = pi[q-1]
    return result

---
## 2. Zaimplementuj testy porównujące szybkość działania wyżej wymienionych algorytmów.

In [6]:
from time import perf_counter

In [7]:
def calculate_time(algorithm, text, pattern):
    start = perf_counter()
    algorithm(text, pattern)
    end = perf_counter()
    print(f"Execution of {algorithm.__name__} is {end - start}")

In [8]:
def compare_time_speed(text, pattern):
    print("*** NAIVE ALGORITHM ***")
    calculate_time(naive_string_matching, text, pattern)
    
    print("*** FA ALGORITHM ***")
    calculate_time(fa_string_matching, text, pattern)
    
    print("*** KMP ALGORITHM ***")
    calculate_time(kmp_string_matching, text, pattern)

## 3. Znajdź wszystkie wystąpienia wzorca "art" w załączonej `ustawie` (1997_714.txt), za pomocą każdego algorytmu.

In [9]:
from io import open

In [10]:
def find_art_in_law(file_name, pattern):
    with open(file_name) as file:
        text = file.read()
        
        # naive algorithm
        result = naive_string_matching(text, pattern)
        print("*** Naive algorithm ***")
        print(f"Number of patterns: {len(result)}")
        print(f"First 10 matches: {result[:10]}")
        
        # fa algorithm
        result = fa_string_matching(text, pattern)
        print("*** Fa algorithm ***")
        print(f"Number of patterns: {len(result)}")
        print(f"First 10 matches: {result[:10]}")
        
        # kmp algorithm
        result = kmp_string_matching(text, pattern)
        print("*** Kmp algorithm ***")
        print(f"Number of patterns: {len(result)}")
        print(f"First 10 matches: {result[:10]}")

In [11]:
find_art_in_law("1997_714.txt", "art")

*** Naive algorithm ***
Number of patterns: 273
First 10 matches: [1156, 1505, 4692, 4734, 4879, 5082, 5148, 5949, 6039, 7266]
*** Fa algorithm ***
Number of patterns: 273
First 10 matches: [1156, 1505, 4692, 4734, 4879, 5082, 5148, 5949, 6039, 7266]
*** Kmp algorithm ***
Number of patterns: 273
First 10 matches: [1156, 1505, 4692, 4734, 4879, 5082, 5148, 5949, 6039, 7266]


## 4. Porównaj szybkość działania algorytmów dla problemu z p. 3.

In [12]:
def time_comparison(file_name, pattern):
    with open(file_name) as file:
        compare_time_speed(file.read(), pattern)

In [13]:
time_comparison("1997_714.txt", "art")

*** NAIVE ALGORITHM ***
Execution of naive_string_matching is 0.03607633600040572
*** FA ALGORITHM ***
Execution of fa_string_matching is 0.020469135999519494
*** KMP ALGORITHM ***
Execution of kmp_string_matching is 0.02616500099975383


## 5. Porównaj szybkość działania algorytmów poprzez wyszukanie słowa "Ukraina" we fragmencie polskiej Wikipedii

In [14]:
time_comparison("passages-head.tsv", "Ukraina")

*** NAIVE ALGORITHM ***
Execution of naive_string_matching is 9.843673806999504
*** FA ALGORITHM ***
Execution of fa_string_matching is 8.22091893599918
*** KMP ALGORITHM ***
Execution of kmp_string_matching is 7.9474486110002545


## 6. Zaproponuj tekst oraz wzorzec, dla którego zmierzony czas działania algorytmów 2 oraz 3 (uwzględniający tylko dopasowanie, bez pre-processingu) będzie co najmniej 5-krotnie krótszy niż dla algorytmu naiwnego.

In [15]:
example_text = "aaa" * 10000000
example_pattern = "a"*100000

In [16]:
compare_time_speed(example_text, example_pattern)

*** NAIVE ALGORITHM ***
Execution of naive_string_matching is 186.1548039900008
*** FA ALGORITHM ***
Execution of fa_string_matching is 8.319026199999826
*** KMP ALGORITHM ***
Execution of kmp_string_matching is 8.533083563999753


## 7. Zaproponuj wzorzec, dla którego zmierzony czas obliczenia tablicy przejścia automatu skończonego będzie co najmniej 5-krotnie dłuższy, niż czas potrzebny na utworzenie funkcji przejścia w algorytmie KMP.

In [17]:
def preprocessing_time_fa(pattern):
    start = perf_counter()
    transition_table(pattern)
    end = perf_counter()
    return end - start

In [18]:
def preprocessing_time_kmp(pattern):
    start = perf_counter()
    prefix_function(pattern)
    end = perf_counter()
    return end - start

In [21]:
prepr_pattern = "qwertyuiopasdfghjkl" * 100

In [22]:
prepr_time_fa = preprocessing_time_fa(prepr_pattern)
prepr_time_kmp = preprocessing_time_kmp(prepr_pattern)

print(f"fa preprocessing time: {preprocessing_time_fa(prepr_pattern)}")
print(f"kmp preprocessing time: {preprocessing_time_kmp(prepr_pattern)}")

if prepr_time_fa/prepr_time_kmp > 5:
    print("kmp preprocessing time is 5 times faster than fa preprocessing time")
else:
    print("kmp preprocessing time is not 5 times faster than fa preprocessing time")

fa preprocessing time: 13.588084934000108
kmp preprocessing time: 0.0003272740004831576
kmp preprocessing time is 5 times faster than fa preprocessing time
